In [99]:
from binance.client import Client
import pandas as pd
import requests
from datetime import datetime, timedelta
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
import os
import warnings
warnings.filterwarnings("ignore")

In [100]:

api_key = 'ZkndW133Kc7JBCDbVV5DV9TSkpkTAm93VaKOSygIDiKi6rOK9tKnOubD4gXY1SSs'
api_secret = 'l3QgD9k6dXDcArEhzSnfXCVtpSykCilnPrk3Ol5xjbJuoH2XYfResrjuAeBHoJgZ'

client = Client(api_key, api_secret)
def convert_to_utc(timestamp_ms):
    """
    Convert the timestamp in milliseconds to a UTC datetime object.
    """
    return datetime.datetime.utcfromtimestamp(timestamp_ms / 1000)


In [101]:
def get_hourly_gold_prices(symbol="PAXGUSDT", interval=Client.KLINE_INTERVAL_1HOUR, days=2):
    """
    Retrieves hourly gold prices and returns them as a DataFrame.
    """
    end_time = datetime.datetime.now().replace(minute=0, second=0, microsecond=0)  # Align to the last full hour
    start_time = end_time - timedelta(days=days)
    start_time_ms = int(start_time.timestamp() * 1000)
    end_time_ms = int(end_time.timestamp() * 1000)

    klines = client.get_historical_klines(symbol, interval, start_time_ms, end_time_ms)

    df = pd.DataFrame(klines, columns=[
        "timestamp", "open", "high", "low", "close", "volume", "close_time",
        "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume",
        "taker_buy_quote_asset_volume", "ignore"
    ])
    
    # Convert timestamps to UTC datetime
    df["timestamp"] = df["timestamp"].apply(convert_to_utc)
    
    df = df[["timestamp", "open", "high", "low", "close", "volume"]]
    df[["open", "high", "low", "close", "volume"]] = df[["open", "high", "low", "close", "volume"]].apply(pd.to_numeric)
    df.rename(columns={"close": "gold_price"}, inplace=True)
    
    return df

In [102]:
def get_hourly_bitcoin_prices(symbol="BTCUSDT", interval=Client.KLINE_INTERVAL_1HOUR, days=2):
    """
    Retrieves hourly Bitcoin prices and returns them as a DataFrame.
    """
    end_time = datetime.datetime.now().replace(minute=0, second=0, microsecond=0)  # Align to the last full hour
    start_time = end_time - timedelta(days=days)
    start_time_ms = int(start_time.timestamp() * 1000)
    end_time_ms = int(end_time.timestamp() * 1000)

    klines = client.get_historical_klines(symbol, interval, start_time_ms, end_time_ms)

    df = pd.DataFrame(klines, columns=[
        "timestamp", "open", "high", "low", "close", "volume", "close_time",
        "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume",
        "taker_buy_quote_asset_volume", "ignore"
    ])
    
    # Convert timestamps to UTC datetime
    df["timestamp"] = df["timestamp"].apply(convert_to_utc)
    
    df = df[["timestamp", "open", "high", "low", "close", "volume"]]
    df[["open", "high", "low", "close", "volume"]] = df[["open", "high", "low", "close", "volume"]].apply(pd.to_numeric)
    df.rename(columns={"close": "bitcoin_price"}, inplace=True)
    
    return df

In [103]:

def convert_to_utc(timestamp_ms):
    """
    Convert the timestamp in milliseconds to a UTC datetime object.
    """
    return datetime.datetime.utcfromtimestamp(timestamp_ms / 1000)

def get_hourly_data_from_binance(symbol='EURUSDT', interval='1h', days=1):
    """
    Retrieves historical hourly data for a symbol from Binance, handling pagination in 20-day chunks.
    """
    # Calculate the end time and the start time based on the number of days
    end_time = datetime.datetime.now().replace(minute=0, second=0, microsecond=0)
    start_time = end_time - timedelta(days=days)
    
    start_time_ms = int(start_time.timestamp() * 1000)
    end_time_ms = int(end_time.timestamp() * 1000)
    
    max_range = 1000  # Binance allows a max of 1000 points per request

    all_hourly_data = []

    # Split the data retrieval into 20-day chunks
    chunk_size = 20  # We fetch 20 days of data at a time
    for i in range(0, days, chunk_size):
        chunk_start_time = start_time + timedelta(days=i)
        chunk_end_time = min(start_time + timedelta(days=(i + chunk_size)), end_time)
        
        chunk_start_time_ms = int(chunk_start_time.timestamp() * 1000)
        chunk_end_time_ms = int(chunk_end_time.timestamp() * 1000)
        
        # Binance API endpoint for historical klines (candlestick data)
        url = f"https://api.binance.com/api/v3/klines"
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': chunk_start_time_ms,
            'endTime': chunk_end_time_ms,
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            # Convert the timestamps to UTC datetime
            hourly_data = [(convert_to_utc(item[0]), float(item[4])) for item in data]  # Add price as the 5th column
            all_hourly_data.extend(hourly_data)
        else:
            print(f"Error: {response.status_code} - {response.text}")
            break
    
    # Convert to DataFrame
    df = pd.DataFrame(all_hourly_data, columns=["timestamp", "price"])
    df.set_index("timestamp", inplace=True)
    
    # Drop any overlapping rows (the first row after each chunk could be a duplicate)
    df = df.loc[~df.index.duplicated(keep='first')]  # Remove duplicates
    
    return df

def calculate_hourly_inflation(symbol='EURUSDT', days=1, interval='1h'):
    """
    Calculates the hourly inflation rates based on hourly exchange rates.
    """
    # Fetch hourly data
    df = get_hourly_data_from_binance(symbol, interval, days)
    
    if df is None or df.empty:
        return None

    # Calculate inflation rates
    hourly_inflation = []
    for i in range(1, len(df)):
        current_time = df.index[i]
        current_rate = df.iloc[i]["price"]
        previous_rate = df.iloc[i - 1]["price"]

        if previous_rate > 0:
            inflation_rate = ((current_rate - previous_rate) / previous_rate) * 100
            hourly_inflation.append((current_time, current_rate, inflation_rate))  # Add both price and inflation rate

    # Create a DataFrame for inflation rates and prices
    inflation_df = pd.DataFrame(hourly_inflation, columns=["timestamp", "price", "inflation_rate"])
    
    # Ensure the timestamp is a regular column (not index)
    inflation_df.reset_index(drop=True, inplace=True)

    return inflation_df


In [104]:

# Fetch data
days=365
gold_prices = get_hourly_gold_prices(days=days)
gold_prices.to_csv("gold_hourly_prices.csv", index=False)

# bitcoin_prices = get_hourly_bitcoin_prices(days=days)
# bitcoin_prices.to_csv("bitcoin_hourly_prices.csv", index=False)

inflation_data = calculate_hourly_inflation(symbol='EURUSDT', days=days)
inflation_data.to_csv("hourly_exchange_rates.csv", index=False)



In [105]:
inflation_file = 'hourly_exchange_rates.csv'
gold_prices_file = 'gold_hourly_prices.csv'
bitcoin_file = 'bitcoin_hourly_prices.csv'

In [106]:
df_inflation = pd.read_csv(inflation_file)
print(df_inflation.head())

             timestamp   price  inflation_rate
0  2024-01-06 14:00:00  1.0883        0.027574
1  2024-01-06 15:00:00  1.0881       -0.018377
2  2024-01-06 16:00:00  1.0881        0.000000
3  2024-01-06 17:00:00  1.0886        0.045952
4  2024-01-06 18:00:00  1.0889        0.027558


In [107]:
df_gold_prices = pd.read_csv(gold_prices_file)
print(df_gold_prices.head())

             timestamp    open    high     low  gold_price    volume
0  2024-01-06 13:00:00  2018.0  2019.0  2016.0      2019.0   14.1820
1  2024-01-06 14:00:00  2018.0  2022.0  2018.0      2021.0    6.8325
2  2024-01-06 15:00:00  2021.0  2022.0  2004.0      2015.0  188.4057
3  2024-01-06 16:00:00  2015.0  2021.0  2014.0      2016.0   27.5479
4  2024-01-06 17:00:00  2017.0  2020.0  2016.0      2017.0    4.6245


In [108]:
final_df = pd.merge(df_inflation, df_gold_prices, on='timestamp', how='inner')
final_df  = pd.DataFrame(final_df)
final_df.drop(columns='price', inplace=True)
print(final_df.head())

             timestamp  inflation_rate    open    high     low  gold_price  \
0  2024-01-06 14:00:00        0.027574  2018.0  2022.0  2018.0      2021.0   
1  2024-01-06 15:00:00       -0.018377  2021.0  2022.0  2004.0      2015.0   
2  2024-01-06 16:00:00        0.000000  2015.0  2021.0  2014.0      2016.0   
3  2024-01-06 17:00:00        0.045952  2017.0  2020.0  2016.0      2017.0   
4  2024-01-06 18:00:00        0.027558  2017.0  2017.0  2015.0      2016.0   

     volume  
0    6.8325  
1  188.4057  
2   27.5479  
3    4.6245  
4    4.4821  


In [109]:
shifted_c = ['inflation_rate', 'open', 'high', 'low', 'volume']
final_df[shifted_c] = final_df[shifted_c].shift(1)
final_df.dropna(inplace=True)
print(final_df.head())


             timestamp  inflation_rate    open    high     low  gold_price  \
1  2024-01-06 15:00:00        0.027574  2018.0  2022.0  2018.0      2015.0   
2  2024-01-06 16:00:00       -0.018377  2021.0  2022.0  2004.0      2016.0   
3  2024-01-06 17:00:00        0.000000  2015.0  2021.0  2014.0      2017.0   
4  2024-01-06 18:00:00        0.045952  2017.0  2020.0  2016.0      2016.0   
5  2024-01-06 19:00:00        0.027558  2017.0  2017.0  2015.0      2013.0   

     volume  
1    6.8325  
2  188.4057  
3   27.5479  
4    4.6245  
5    4.4821  


In [110]:
os.environ["HOPSWORKS_API_KEY"] = "WMg6iOTOV75YfUcb.AcuNGWjcMS79s4bz0FqZsRaJtcU8Siqu7Mx084dfkRHauMxK251VTYLYSBAIqBb5"
project = hopsworks.login()


2025-01-05 14:35:48,338 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-05 14:35:48,342 INFO: Initializing external client
2025-01-05 14:35:48,342 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 14:35:49,688 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1168537


In [111]:
fs = project.get_feature_store() 

In [112]:
data_historic = fs.get_or_create_feature_group(
    name='data_historic',
    description='historic data all features',
    version=1,
    primary_key=['timestamp'],
)

In [113]:
data_historic.insert(final_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1168537/fs/1159240/fg/1394572


Uploading Dataframe: 100.00% |██████████| Rows 8759/8759 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: data_historic_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1168537/jobs/named/data_historic_1_offline_fg_materialization/executions


(Job('data_historic_1_offline_fg_materialization', 'SPARK'), None)